In [2]:
!nvidia-smi
!pip -q install ultralytics opencv-python lxml tqdm pyyaml


Sun Feb  1 12:30:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             50W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!unzip archive.zip

Archive:  archive.zip
   creating: archive/
  inflating: __MACOSX/._archive      
   creating: archive/images/
  inflating: __MACOSX/archive/._images  
   creating: archive/annotations/
  inflating: __MACOSX/archive/._annotations  
  inflating: archive/images/potholes87.png  
  inflating: __MACOSX/archive/images/._potholes87.png  
  inflating: archive/images/potholes297.png  
  inflating: __MACOSX/archive/images/._potholes297.png  
  inflating: archive/images/potholes93.png  
  inflating: __MACOSX/archive/images/._potholes93.png  
  inflating: archive/images/potholes283.png  
  inflating: __MACOSX/archive/images/._potholes283.png  
  inflating: archive/images/potholes78.png  
  inflating: __MACOSX/archive/images/._potholes78.png  
  inflating: archive/images/potholes268.png  
  inflating: __MACOSX/archive/images/._potholes268.png  
  inflating: archive/images/potholes532.png  
  inflating: __MACOSX/archive/images/._potholes532.png  
  inflating: archive/images/potholes44.png  
  inflat

In [4]:
import random, shutil
from pathlib import Path
from xml.etree import ElementTree as ET

random.seed(42)

SRC_ROOT = Path("/content/archive")
# adjust if your dataset structure differs
IMG_DIR = SRC_ROOT / "images"
ANN_DIR = SRC_ROOT / "annotations"

OUT_ROOT = Path("/content/pothole_yolo")
CLASSES = ["pothole"]

def voc_to_yolo_bbox(size_w, size_h, xmin, ymin, xmax, ymax):
    xmin = max(0, min(xmin, size_w - 1))
    ymin = max(0, min(ymin, size_h - 1))
    xmax = max(0, min(xmax, size_w - 1))
    ymax = max(0, min(ymax, size_h - 1))
    xc = (xmin + xmax) / 2.0 / size_w
    yc = (ymin + ymax) / 2.0 / size_h
    bw = (xmax - xmin) / size_w
    bh = (ymax - ymin) / size_h
    return xc, yc, bw, bh

def parse_voc(xml_path: Path):
    root = ET.parse(xml_path).getroot()
    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    objs = []
    for obj in root.findall("object"):
        name = obj.find("name").text.strip()
        if name not in CLASSES:
            continue
        b = obj.find("bndbox")
        xmin = int(float(b.find("xmin").text))
        ymin = int(float(b.find("ymin").text))
        xmax = int(float(b.find("xmax").text))
        ymax = int(float(b.find("ymax").text))
        objs.append((name, (w, h, xmin, ymin, xmax, ymax)))
    return objs

# Pair xml with image by stem
pairs = []
for xml in sorted(ANN_DIR.glob("*.xml")):
    stem = xml.stem
    img = None
    for ext in [".jpg", ".jpeg", ".png"]:
        p = IMG_DIR / f"{stem}{ext}"
        if p.exists():
            img = p
            break
    if img:
        pairs.append((img, xml))

assert pairs, "No (image, xml) pairs found. Check IMG_DIR / ANN_DIR."

random.shuffle(pairs)
n = len(pairs)
n_train = int(0.8 * n)
n_val = int(0.1 * n)

splits = {
    "train": pairs[:n_train],
    "val": pairs[n_train:n_train + n_val],
    "test": pairs[n_train + n_val:],
}

for s in splits:
    (OUT_ROOT / "images" / s).mkdir(parents=True, exist_ok=True)
    (OUT_ROOT / "labels" / s).mkdir(parents=True, exist_ok=True)

for split, items in splits.items():
    for img_path, xml_path in items:
        shutil.copy2(img_path, OUT_ROOT / "images" / split / img_path.name)
        out_lbl = OUT_ROOT / "labels" / split / f"{img_path.stem}.txt"
        objs = parse_voc(xml_path)
        with out_lbl.open("w") as f:
            for name, (w, h, xmin, ymin, xmax, ymax) in objs:
                cls_id = CLASSES.index(name)
                xc, yc, bw, bh = voc_to_yolo_bbox(w, h, xmin, ymin, xmax, ymax)
                if bw <= 0 or bh <= 0:
                    continue
                f.write(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

data_yaml = OUT_ROOT / "data.yaml"
data_yaml.write_text(f"""path: {OUT_ROOT}
train: images/train
val: images/val
test: images/test
names:
  0: pothole
""")
print("Prepared:", data_yaml)


Prepared: /content/pothole_yolo/data.yaml


In [9]:
!yolo detect train \
  model=yolov8s.pt \
  data=/content/pothole_yolo/data.yaml \
  imgsz=768 \
  epochs=120 \
  batch=16 \
  device=0 \
  workers=8 \
  cache=True \
  optimizer=AdamW \
  lr0=0.003 \
  cos_lr=True \
  patience=25 \
  warmup_epochs=3 \
  hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 \
  degrees=5 translate=0.1 scale=0.5 shear=0.0 perspective=0.0 \
  flipud=0.0 fliplr=0.5 \
  mosaic=1.0 mixup=0.1 \
  close_mosaic=15

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=15, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/pothole_yolo/data.yaml, degrees=5, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=

In [6]:
!yolo detect val model=/content/runs/detect/train/weights/best.pt data=/content/pothole_yolo/data.yaml split=test imgsz=768

Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 93 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2923.6±669.0 MB/s, size: 480.8 KB)
val: Scanning /content/pothole_yolo/labels/test... 67 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 67/67 491.2it/s 0.1s
val: New cache created: /content/pothole_yolo/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 3.3it/s 1.5s
                   all         67        161      0.798      0.677      0.779       0.52
Speed: 4.0ms preprocess, 7.2ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /content/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [5]:
!yolo detect train \
  model=yolov8m.pt \
  data=/content/pothole_yolo/data.yaml \
  imgsz=768 \
  epochs=80 \
  batch=8 \
  device=0 \
  cache=True \
  optimizer=AdamW \
  lr0=0.002 \
  cos_lr=True \
  patience=15 \
  close_mosaic=15

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=15, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/pothole_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=